In [1]:
import libcst

In [2]:
import lolviz

In [3]:
a = libcst.parse_module("def foo(): pass")

In [4]:
a

Module(
    body=[
        FunctionDef(
            name=Name(
                value='foo',
                lpar=[],
                rpar=[],
            ),
            params=Parameters(
                params=[],
                star_arg=MaybeSentinel.DEFAULT,
                kwonly_params=[],
                star_kwarg=None,
                posonly_params=[],
                posonly_ind=MaybeSentinel.DEFAULT,
            ),
            body=SimpleStatementSuite(
                body=[
                    Pass(
                        semicolon=MaybeSentinel.DEFAULT,
                    ),
                ],
                leading_whitespace=SimpleWhitespace(
                    value=' ',
                ),
                trailing_whitespace=TrailingWhitespace(
                    whitespace=SimpleWhitespace(
                        value='',
                    ),
                    comment=None,
                    newline=Newline(
                        value=None,
          

In [5]:
class ImportVisitor(libcst.CSTVisitor):
    def __init__(self):
        self.inImport = False
        self.importNames = set()
        self.otherNames = set()

    def visit_Import(self, node):
        self.inImport = True

    def leave_Import(self, node):
        self.inImport = False

    def leave_Name(self, node):
        if self.inImport:
            self.importNames.add(node.value)
        else:
            self.otherNames.add(node.value)
            

In [6]:
text = """\
import abc
import foo
abc.func(xyz)"""

In [7]:
code = libcst.parse_module(text)

In [8]:
visitor = ImportVisitor()

In [9]:
temp = code.visit(visitor)

In [10]:
visitor.importNames

{'abc', 'foo'}

In [11]:
visitor.otherNames

{'abc', 'func', 'xyz'}

In [12]:
visitor.importNames - visitor.otherNames

{'foo'}

In [13]:
class FuncVarVisitor(libcst.CSTVisitor):
    def __init__(self):
        self.current = list()
        self.func_var = dict()
    def visit_FunctionDef(self, node):
        name = node.name.value
        self.current.append(name)
        self.func_var["::".join(self.current)] = set()
    def leave_FunctionDef(self, node):
        self.current.pop()
    def leave_Name(self, node):
        if self.current:
            func = self.current[-1]
            name = node.value
            self.func_var["::".join(self.current)].add(name)
            

In [14]:
code = libcst.parse_module("""\
def foo():
    a = 10
    return a * 20

def bar():
    def inner():
        var = 20
        return var
    buz = 1
    return buz + 1

foo()
bar()""")



In [15]:
func_var = FuncVarVisitor()

In [16]:
temp = code.visit(func_var)

In [17]:
func_var.func_var

{'foo': {'a', 'foo'}, 'bar': {'bar', 'buz'}, 'bar::inner': {'inner', 'var'}}

In [18]:
import re

In [19]:
re.sub(r"\d{3}", "abc", "a123d4567")

'aabcdabc7'

In [20]:
re.sub(r"\b[a-z]", lambda m: m.group(0).upper(), "abc def 1ff")

'Abc Def 1ff'

In [21]:
class Snake2Camel(libcst.CSTTransformer):
    def leave_FunctionDef(self, node, updated):
        name = updated.name.value
        camel_name = re.sub(r"_([a-z])", lambda m: m.group(1).upper(), name)
        return libcst.FunctionDef(name=libcst.Name(value=camel_name), params=updated.params, body=updated.body)

In [22]:
snake = libcst.parse_module("""\
def foo_bar():
    a = 10
    return a * 20

def bar_buz():
    def inner():
        var = 20
        return var
    buz = 1
    return buz + 1
""")

In [23]:
snake2camel = Snake2Camel()

In [24]:
camel = snake.visit(snake2camel)

In [25]:
print(camel.code)

def fooBar():
    a = 10
    return a * 20
def barBuz():
    def inner():
        var = 20
        return var
    buz = 1
    return buz + 1

